# Arbitrage between cryptocurrencies centralized exchanges

## Summary

Sometimes the same cryptocurrency is priced differently in different exchanges. We could take advantage of this by buying on exchanges where the price is the lowest and selling on the exchange where the price is the highest. To find this opprtunities we need to track the price differences between pairs of exchanges and then execute the orders.

But having a price difference doesn't necessarily mean we can make money from it. Indeed there are commissions cost, time constraints and transfer between exchanges fees depending on the methods you use.There are two ways to do this and each has its pros and cons.

To make the explanation easier we will create a context . Let C be a cryptocurrency, and there is currently an arbitrage opportunity for C. A is the exchange where the price of C is the lowest and B the exchange where the price of C is the highest. Now for executing the orders we can use one of the methods mentioned earlier

### 1. Transfer Method <br>
We buy C on exchange A, transfer it to B, then sell it on exchange B.

    - Pros: We don't need to have in advance cryptocurrencies on exchanges A and B (unlike the 2nd method we will present later)
    - Cons: The transfer fees are usually much higher than the commission fee for buying and selling on each exchanges and those transfer fees are usually different per cryptocurrency which will require much maintenance. This high fees are less problematic when you invest big amounts of money (> 50 000$). But even by investing big amounts there is another constraint: the transfer can take a long time (more than 10 minutes) which can make the opportunity vanish.
    - How can we reduce the cons: Look for cryptocurrency with low transfer fees and fast transfer
    
### 2. Parallel Buy and Sell Method <br>
Before tracking arbitrage opportunities, we buy some cryptocurrencies on exchange B. When an arbitrage opportunity occurs, we buy cryptocurrencies on exchange A and sell the cryptocurrencies we had bought before on exchange B. How can we earn more with this method? Let's put it in perspective. Let's suppose the price of 1 C is 200$\$$ and it's the same on exchanges A and B. And that we have 1000$\$$ on exchange A, and 5 C on exchange B. So we have this state at the beginning

| A | B | Total |
| :-: | :-: | :-: |
| 1000$\$$ | 5 C | 1000$\$$ and 5 C |

Then an arbitrage opportunities occurs, price of 1 C becomes 250$\$$ on exchange B. So we buy 5 C on exchange A with the 1000$\$$ and sell the 5C we have on exchange B because it's more beneficial to buy on exchange A and to sell on exchange B <br>

| A	| B	| Total |
| :-: | :-: | :-: |
| 5 C |	1250$\$$ | 1250$\$$ and 5C |<br>

So we can see that our total increased. Our initial equity was 1000$\$$ and 5 C ~= 2000$\$$ and our final equity is 1250$\$$ and 5 C ~= 2250$\$$

    - Pros: No need to transfer between exchanges anymore
    - Cons: The mains con is that the exchanges were we buy and sell are the dependant on wether we have money or cryptocurrencies on this exchanges. For example in our final state, we can only buy on exchange B and sell on exchange, meaning that in the next arbitrage opportunity we can catch will be when price on exchange B is lower than price on exchange A. If the price differences are balanced between exchanges (the number of time price on A is lower than price on B is approximately equals to the number of time price on B is lower than price of A), this can work well but otherwise, we will miss a lot of opportunities
    - How can we reduce the cons: We can find out which cryptocurrencies have balanced price differences and only execute our check on them. <br>
    
### Final method choice
Method 2 doesn't need big amount of money or doesn't have time constraints, so it's fit better our needs for now and we will explore this method.

## Let's get our hands dirty

## Imports

In [75]:
from feed.loader import CsvLoader
from models.asset import Asset
import pandas as pd
import numpy as np

In [76]:
SYMBOL_TO_ARBITRAGE = "XRPUSDT",
BINANCE_EXCHANGE = "BINANCE"
KUCOIN_EXCHANGE = "KUCOIN"
BINANCE_ASSET = Asset(symbol=SYMBOL_TO_ARBITRAGE, exchange=BINANCE_EXCHANGE)
KUCOIN_ASSET = Asset(symbol=SYMBOL_TO_ARBITRAGE, exchange=KUCOIN_EXCHANGE)
BINANCE_FEE = 0.001
KUCOIN_FEE = 0.001
MARGIN_FACTOR = 1

In [77]:
csv_filenames = {
    BINANCE_ASSET: f"data/xrpusdt_one_week_binance.csv",
    KUCOIN_ASSET: f"data/xrpusdt_one_week_kucoin.csv"
}

csv_loader = CsvLoader(csv_filenames)
csv_loader.load()
dataframes = csv_loader.candle_dataframes

## Create a function to check an arbitrage opportunity

In [78]:
def check_arbitrage_opportunity(
        binance_price_action: pd.DataFrame,
        kucoin_price_action: pd.DataFrame,
        margin_factor: float = 1,
) -> bool:
    binance_prices = binance_price_action["close"]
    kucoin_prices = kucoin_price_action["close"]

    binance_fee = binance_prices * BINANCE_FEE
    kucoin_fee = kucoin_prices * KUCOIN_FEE
    total_fee = binance_fee + kucoin_fee
    price_diff = abs(binance_prices - kucoin_prices)
    return (
            (price_diff > margin_factor * total_fee)
            & (binance_price_action["volume"] != 0)
            & (kucoin_price_action["volume"] != 0)
    )

In [79]:
binance_df = dataframes[BINANCE_ASSET]
kucoin_df = dataframes[KUCOIN_ASSET]
binance_price_action = binance_df[["close", "volume"]].apply(pd.to_numeric)
kucoin_price_action = kucoin_df[["close", "volume"]].apply(pd.to_numeric)
arbitrages_opportunities_df = binance_price_action.rename(
    columns={"close": "binance_price", "volume": "binance_volume"}, inplace=False
)
arbitrages_opportunities_df[["kucoin_price", "kucoin_volume"]] = kucoin_price_action
arbitrages_opportunities_df["Arbitrage opportunity?"] = np.where(
    check_arbitrage_opportunity(binance_price_action, kucoin_price_action),
    True,
    False,
)
arbitrages_opportunities_df = arbitrages_opportunities_df[
    arbitrages_opportunities_df["Arbitrage opportunity?"]
]
arbitrages_opportunities_df["Side"] = np.where(
    arbitrages_opportunities_df["binance_price"]
    > arbitrages_opportunities_df["kucoin_price"],
    "Binance",
    "Kucoin",
)

In [80]:
arbitrages_opportunities_df

,binance_price,binance_volume,kucoin_price,kucoin_volume,Arbitrage opportunity?,Side
timestamp,,,,,,
2021-06-22 20:20:00+00:00,0.57180,24747.616315,0.5733,800884.90,True,Kucoin
2021-06-23 11:30:00+00:00,0.63500,10790.911506,0.6331,295397.10,True,Binance
2021-06-24 12:00:00+00:00,0.65649,27401.409809,0.6579,1087473.87,True,Kucoin
2021-06-25 03:38:00+00:00,0.67697,2053.590026,0.6755,380618.10,True,Binance
2021-06-26 08:47:00+00:00,0.58809,1663.173614,0.5869,451598.35,True,Binance
2021-06-26 12:18:00+00:00,0.60244,267377.574098,0.6038,341983.98,True,Kucoin


## Conclusion

We can see that we have 6 arbitrage opportunities in 1 week which is not bad with quite a good balance in the case of XRP. Now it's time to backtest this to see whether it's real or not.